In [31]:
!pip install requests
!pip install pandas
!pip install mp-api
!pip install requests-html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 443.0 kB/s eta 0:00:000:00:01 eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 216.8 kB/s eta 0:00:001m257.8 kB/s eta 0:00:01


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 506.7 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 266.6 kB/s eta 0:00:00m eta 0:00:010:00:01
Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
Using legacy 'setup.py install' for parse, since package 'wheel' is not installed.
  Running setup.py install for parse ... done
  Running setup.py install for bs4 ... done


In [14]:
import csv
import json
import os
import pandas as pd
from mp_api.client import MPRester
from tqdm.notebook import tqdm
from ast import literal_eval
import math
from concurrent.futures import ThreadPoolExecutor
import requests
from requests_html import HTMLSession
import random

In [85]:
ROOT_DIR = os.path.abspath("")
DATA_DIR = os.path.join(ROOT_DIR, 'CrabNet/data/materials_data')
TRAINING_CSV_FILE = os.path.join(ROOT_DIR, "Vacancyenergydata.csv")
API_KEY = "3XX6x9Nd3cKWf9Q4wpaio64CBHFp2uDq"
TRAINING_DATA = []
mpr = MPRester(API_KEY)

print(TRAINING_CSV_FILE)

/home/murimikelvin/ML-perovskites-project/Vacancyenergydata.csv


## Step1:Collecting data.

#### Load Training data.

In [102]:
df = pd.read_csv(TRAINING_CSV_FILE)
print(df.columns)
df = df[['Chemical formula', 'Vacancy energy [eV/O atom]']]
new_names = {
    'Chemical formula': 'chem_formula',
    'Vacancy energy [eV/O atom]': 'e_ovac'
}
df.rename(columns=new_names, inplace=True)
df = df.to_dict("records")
df[:10]


Index(['Chemical formula', 'A', 'B', 'In literature', 'Valence A', 'Valence B',
       'Radius A [ang]', 'Radius B [ang]', 'Lowest distortion',
       'Formation energy [eV/atom]', 'Stability [eV/atom]',
       'Magnetic moment [mu_B]', 'Volume per atom [A^3/atom]', 'Band gap [eV]',
       'a [ang]', 'b [ang]', 'c [ang]', 'alpha [deg]', 'beta [deg]',
       'gamma [deg]', 'Vacancy energy [eV/O atom]'],
      dtype='object')


[{'chem_formula': 'Ac2O3', 'e_ovac': '3.150'},
 {'chem_formula': 'AcAgO3', 'e_ovac': '0.817'},
 {'chem_formula': 'AcAlO3', 'e_ovac': '6.695'},
 {'chem_formula': 'AcAsO3', 'e_ovac': '3.634'},
 {'chem_formula': 'AcAuO3', 'e_ovac': '0.807'},
 {'chem_formula': 'AcBO3', 'e_ovac': '4.163'},
 {'chem_formula': 'AcBaO3', 'e_ovac': '-1.771'},
 {'chem_formula': 'AcBeO3', 'e_ovac': '-0.080'},
 {'chem_formula': 'AcBiO3', 'e_ovac': '2.653'},
 {'chem_formula': 'AcCaO3', 'e_ovac': '-1.025'}]

#### Loading prediction data

In [36]:
list_of_available_fields = mpr.summary.available_fields

print(dir(mpr.summary.search))

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [8]:

# mpr.materials.available_fields

fields = ['nelements', 'formula_pretty', 'formula_anonymous', 'composition', 'composition_reduced', 'icsd_ids']

docs = mpr.summary.search(elements=["O"], fields=fields)



Retrieving SummaryDoc documents: 100%|████████████████████████████████████████████████████████████████████████| 81887/81887 [01:05<00:00, 1250.78it/s]


In [9]:
print(len(docs))

81887


## Step 2: Data cleaning

#### Cleaning training data

In [103]:
TRAINING_DATA = [m for m in df if m['e_ovac'] != '-']
PREDICTION_DATA = [m for m in df if m['e_ovac'] == '-']



print("Materials in the csv file without E_Ovac: ", len(PREDICTION_DATA))
print("Materials in the csv file with E_Ovac: ", len(TRAINING_DATA))
print("Total materials in the csv file: ", len(df))


Materials in the csv file without E_Ovac:  415
Materials in the csv file with E_Ovac:  4914
Total materials in the csv file:  5329


In [104]:
print(TRAINING_DATA[0])

{'chem_formula': 'Ac2O3', 'e_ovac': '3.150'}


In [105]:
print(PREDICTION_DATA[0])

{'chem_formula': 'AcMgO3', 'e_ovac': '-'}


#### Create Model Training Data (Create train.csv, val.csv, test.csv)

In [106]:
## Split the data into train, test, val

random.shuffle(TRAINING_DATA) # Shuffle the training data list on every run
split_ratio = "6:2:2" # Set the ratio at with to split the data into train, test and validation sets from the training data list

ratios = [round(int(r)/10 * len(TRAINING_DATA)) for r in split_ratio.split(':')] # Get the number of materials from the training data list according tp the ratio

# Get X amount of data from the training dataset accoring to the set rratio
train_set = TRAINING_DATA[:ratios[0]] 
test_set = TRAINING_DATA[ratios[0]:(ratios[0] + ratios[1])]
val_set = TRAINING_DATA[(ratios[0] + ratios[1]):]

print(len(train_set), len(test_set), len(val_set))
print(ratios, sum(ratios), len(TRAINING_DATA))

print(train_set[:5])
print(test_set[:5])
print(val_set[:5])
# print(TRAINING_DATA[:5])

2948 983 983
[2948, 983, 983] 4914 4914
[{'chem_formula': 'RbTmO3', 'e_ovac': '-0.698'}, {'chem_formula': 'YbPmO3', 'e_ovac': '-1.616'}, {'chem_formula': 'UPuO3', 'e_ovac': '3.167'}, {'chem_formula': 'MgYbO3', 'e_ovac': '-4.027'}, {'chem_formula': 'TcHfO3', 'e_ovac': '1.949'}]
[{'chem_formula': 'CeGdO3', 'e_ovac': '4.348'}, {'chem_formula': 'TmBiO3', 'e_ovac': '0.344'}, {'chem_formula': 'TlLuO3', 'e_ovac': '-2.715'}, {'chem_formula': 'NpBeO3', 'e_ovac': '1.432'}, {'chem_formula': 'TcRuO3', 'e_ovac': '1.727'}]
[{'chem_formula': 'IrVO3', 'e_ovac': '-2.370'}, {'chem_formula': 'CdErO3', 'e_ovac': '-2.290'}, {'chem_formula': 'Dy2O3', 'e_ovac': '0.855'}, {'chem_formula': 'PtBiO3', 'e_ovac': '-4.385'}, {'chem_formula': 'PuYbO3', 'e_ovac': '-2.625'}]


In [107]:
## Save data to CSV

fields = ['formula', 'target']
trin_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in train_set]
val_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in val_set]
test_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in test_set]

# print(rows)
## Create train.csv with train_set data
with open(os.path.join(DATA_DIR, 'train.csv'), 'w') as train_file:
    csvwriter = csv.writer(train_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(trin_rows)

## Create val.csv with train_set data
with open(os.path.join(DATA_DIR, 'val.csv'), 'w') as val_file:
    csvwriter = csv.writer(val_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(val_rows)

## Create test.csv with train_set data
with open(os.path.join(DATA_DIR, 'test.csv'), 'w') as test_file:
    csvwriter = csv.writer(test_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(test_rows)
    

#### Train the crabnet model